# Importing Needed Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('whitegrid')
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")

from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Dense, LSTM, Dropout

from sklearn.preprocessing import MinMaxScaler

# Reading The Data

In [ ]:
data_dir = '../input/bitcoin-prediction-dataset-bullrun/BTC_USD_Price_Prediction_Data.csv'
df = pd.read_csv(data_dir)

# Data Exploration

In [ ]:
df.head()

In [ ]:
df.info()

# Set Date column as an index

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace= True)

In [ ]:
plt.figure(figsize=(15, 6))
df[r'24h Open (USD)'].plot()
df[r'Closing Price (USD)'].plot()
plt.ylabel(None)
plt.xlabel(None)
plt.title("Opening & Closing Price of BitCoin")
plt.legend(['Open Price', 'Close Price'])
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15, 6))
df[r'24h High (USD)'].plot()
df[r'24h Low (USD)'].plot()
plt.ylabel(None)
plt.xlabel(None)
plt.title("High & Low Price of BitCoin in 24h")
plt.legend(['High Price', 'Low Price'])
plt.tight_layout()
plt.show()

# Data Preprocessing

**Choosing Prediction Column**

In [ ]:
n_cols = 1
dataset = df[r"Closing Price (USD)"]
dataset = pd.DataFrame(dataset)
data = dataset.values

data.shape

**Normalizing Data**

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range= (0, 1))
scaled_data = scaler.fit_transform(np.array(data))

**Splitting the Data**

In [ ]:
# 75% to Train , 25% to Test
train_size = int(len(data) * 0.75)
test_size = len(data) - train_size
print("Train Size :",train_size,"Test Size :",test_size)

In [ ]:
train_data = scaled_data[0:train_size, :]
train_data.shape

# Creating training set

In [ ]:
# Creating a Training set with 60 time-steps
x_train = []
y_train = []
time_steps = 60
n_cols = 1

for i in range(time_steps, len(train_data)):
    x_train.append(train_data[i-time_steps:i, :n_cols])
    y_train.append(train_data[i, :n_cols])
    if i<=time_steps:
        print('X_train: ', x_train)
        print('y_train:' , y_train)

In [ ]:
# Convert to numpy array
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
# Reshaping the input to (n_samples, time_steps, n_feature)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], n_cols))

In [ ]:
x_train.shape , y_train.shape

# LSTM Model

**Model Structure**

In [ ]:
model = Sequential([
    LSTM(50, return_sequences= True, input_shape= (x_train.shape[1], n_cols)),
    LSTM(64, return_sequences= False),
    Dense(32),
    Dense(16),
    Dense(n_cols)
])

model.compile(optimizer= 'adam', loss= 'mse' , metrics= "mean_absolute_error")
model.summary()


In [ ]:
# Fitting the LSTM to the Training set
history = model.fit(x_train, y_train, epochs= 100, batch_size= 32)

# Model Evaluation

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(history.history["loss"])
plt.plot(history.history["mean_absolute_error"])
plt.legend(['Mean Squared Error','Mean Absolute Error'])
plt.title("Losses")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()
    

# Predection

# Creating testing set

In [ ]:
# Creating a testing set with 60 time-steps and 1 output
time_steps = 60
test_data = scaled_data[train_size - time_steps:, :]

x_test = []
y_test = []
n_cols = 1

for i in range(time_steps, len(test_data)):
    x_test.append(test_data[i-time_steps:i, 0:n_cols])
    y_test.append(test_data[i, 0:n_cols])
x_test, y_test = np.array(x_test), np.array(y_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], n_cols))

In [ ]:
x_test.shape , y_test.shape

In [ ]:
# Get Prediction
predictions = model.predict(x_test)

In [ ]:
#inverse predictions scaling
predictions = scaler.inverse_transform(predictions)
predictions.shape

**Root mean square error**

In [ ]:
#inverse y_test scaling
y_test = scaler.inverse_transform(y_test)

RMSE = np.sqrt(np.mean( y_test - predictions )**2).round(2)
RMSE

In [ ]:
preds_acts = pd.DataFrame(data={'Predictions':predictions.flatten(), 'Actuals':y_test.flatten()})
preds_acts

If you found it helpful, do upvote

Feel free to comment

I would love to have suggestions.